# Xebicon '19

Disponibilité des Velib dans la ville de Paris.

Basé surl'OpenData de la ville Paris : https://opendata.paris.fr/pages/home/

En utilisant le dataset Velib datasets : https://opendata.paris.fr/explore/dataset/velib-disponibilite-en-temps-reel/information/

In [2]:
#!pip install -r ../requirements.txt

In [12]:
import pandas as pd

In [13]:
#stations = pd.read_csv("https://opendata.paris.fr/api/v2/catalog/datasets/velib-disponibilite-en-temps-reel/exports/csv?rows=-1&timezone=UTC&delimiter=%3B", delimiter=";")
stations = pd.read_csv("http://minio.nas.helyx.io:9000/smovengo/2019/11/03/23/2019-11-03-23-19.csv", delimiter=";")

URLError: <urlopen error [Errno 111] Connection refused>

In [3]:
stations[stations['station_name'].str.contains("Mongolfier")]

,nbfreeedock,nbbikeoverflow,nbedock,overflowactivation,densitylevel,creditcard,station,maxbikeoverflow,kioskstate,nbfreedock,...,overflow,nbdock,nbebike,nbebikeoverflow,station_code,station_name,station_state,station_type,geo,duedate
919,11,0,39,no,0,no,"{""code"": ""44101"", ""name"": ""Place Mongolfier"", ...",0,yes,0,...,no,0,12,0,44101,Place Mongolfier,Operative,yes,"48.8151600301, 2.45928794146",2018-10-30


In [14]:
stations.describe()

,nbfreeedock,nbbikeoverflow,nbedock,densitylevel,maxbikeoverflow,nbfreedock,nbbike,nbdock,nbebike,nbebikeoverflow,station_code,lat,lng
count,1391.000000,1391.000000,1391.000000,1391.000000,1391.000000,1391.000000,1391.000000,1391.000000,1391.000000,1391.000000,1391.000000,1391.000000,1391.000000
mean,15.846873,0.089863,25.327103,0.902229,16.213515,3.619698,5.608196,5.605320,5.297628,0.010784,18043.317757,48.857524,2.342391
std,12.636204,0.403017,15.979892,0.297112,18.328725,9.012534,5.637073,12.945439,4.172778,0.122440,12066.252504,0.029009,0.054809
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1001.000000,48.765050,2.165597
25%,6.000000,0.000000,17.000000,1.000000,0.000000,0.000000,2.000000,0.000000,2.000000,0.000000,11003.500000,48.837992,2.306331
50%,15.000000,0.000000,25.000000,1.000000,10.000000,0.000000,4.000000,0.000000,4.000000,0.000000,16011.000000,48.858103,2.344577
75%,23.000000,0.000000,35.000000,1.000000,30.000000,1.000000,8.000000,1.000000,7.000000,0.000000,21161.000000,48.878107,2.378829
max,67.000000,11.000000,73.000000,1.000000,71.000000,64.000000,48.000000,70.000000,24.000000,2.000000,92008.000000,48.947022,2.538242


In [5]:
stations[stations['nbebike'] == stations.describe()['nbebike']['max']]

,nbfreeedock,nbbikeoverflow,nbedock,overflowactivation,densitylevel,creditcard,station,maxbikeoverflow,kioskstate,nbfreedock,...,overflow,nbdock,nbebike,nbebikeoverflow,station_code,station_name,station_state,station_type,geo,duedate
1048,12,0,53,no,1,yes,"{""code"": ""11001"", ""name"": ""Richard Lenoir - Pl...",59,yes,1,...,no,1,24,0,11001,Richard Lenoir - Place de la Bastille,Operative,yes,"48.8538327, 2.3697222",2019-05-24


In [6]:
stations[['lat','lng']] = stations['geo'].str.split(',',expand=True)
stations[['lat','lng', 'nbfreeedock']]
stations = stations.astype({'lng': 'float64', 'lat': 'float64'})
stations.dtypes

nbfreeedock             int64
nbbikeoverflow          int64
nbedock                 int64
overflowactivation     object
densitylevel            int64
creditcard             object
station                object
maxbikeoverflow         int64
kioskstate             object
nbfreedock              int64
nbbike                  int64
overflow               object
nbdock                  int64
nbebike                 int64
nbebikeoverflow         int64
station_code            int64
station_name           object
station_state          object
station_type           object
geo                    object
duedate                object
lat                   float64
lng                   float64
dtype: object

In [7]:
stations.columns

Index(['nbfreeedock', 'nbbikeoverflow', 'nbedock', 'overflowactivation',
       'densitylevel', 'creditcard', 'station', 'maxbikeoverflow',
       'kioskstate', 'nbfreedock', 'nbbike', 'overflow', 'nbdock', 'nbebike',
       'nbebikeoverflow', 'station_code', 'station_name', 'station_state',
       'station_type', 'geo', 'duedate', 'lat', 'lng'],
      dtype='object')

Configure Google Maps API Key: https://jupyter-gmaps.readthedocs.io/en/latest/authentication.html

In [16]:
import os
import gmaps
import gmaps.datasets
import IPython
gmaps.configure(api_key=os.environ["GOOGLE_API_KEY"])  # Your Google API key

#print("Google Api Key:", os.environ["GOOGLE_API_KEY"])

In [17]:
paris_coordinates = (48.8566, 2.3522)

In [18]:
stations_with_almost_no_ebikes = stations[stations['nbebike'] < 4]

fig = gmaps.figure(center=paris_coordinates, zoom_level=13)

almost_no_ebikes_heatmap_layer = gmaps.heatmap_layer(stations_with_almost_no_ebikes[['lat', 'lng']])#
almost_no_ebikes_heatmap_layer.gradient = [(200, 200, 200, 0.0), 'yellow', 'red']
#almost_no_ebikes_heatmap_layer.max_intensity = 1
almost_no_ebikes_heatmap_layer.point_radius = 32

fig.add_layer(almost_no_ebikes_heatmap_layer)
#fig.add_layer(gmaps.bicycling_layer())

fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
stations_with_many_ebikes = stations[stations['nbebike'] > 5]

fig = gmaps.figure(center=paris_coordinates, zoom_level=13)

many_ebikes_heatmap_layer = gmaps.heatmap_layer(stations_with_many_ebikes[['lat', 'lng']])#
many_ebikes_heatmap_layer.gradient = [(200, 200, 200, 0.0), 'yellow', 'green']
#many_ebikes_heatmap_layer.max_intensity = 2
many_ebikes_heatmap_layer.point_radius = 32

fig.add_layer(many_ebikes_heatmap_layer)
#fig.add_layer(gmaps.bicycling_layer())

fig

Figure(layout=FigureLayout(height='420px'))

In [12]:

fig = gmaps.figure(center=paris_coordinates, zoom_level=13)

fig.add_layer(almost_no_ebikes_heatmap_layer)
fig.add_layer(many_ebikes_heatmap_layer)
#fig.add_layer(gmaps.bicycling_layer())

fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
paris_coordinates = (48.8566, 2.3522)

fig = gmaps.figure(center=paris_coordinates, zoom_level=13)
heatmap_layer = gmaps.heatmap_layer(stations[['lat', 'lng']], weights=stations['nbebike'])
heatmap_layer.gradient = [(200, 200, 200, 0.0), 'yellow', 'green']
heatmap_layer.point_radius = 32

fig.add_layer(heatmap_layer)
fig.add_layer(gmaps.bicycling_layer())

fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
fig = gmaps.figure(center=paris_coordinates, zoom_level=13)
heatmap_layer = gmaps.heatmap_layer(stations[['lat', 'lng']], weights=stations['nbfreeedock'])
heatmap_layer.gradient = [(200, 200, 200, 0.0), 'yellow', 'green']
heatmap_layer.point_radius = 32

print(heatmap_layer.gradient)
fig.add_layer(heatmap_layer)
fig.add_layer(gmaps.bicycling_layer())

fig

['rgba(200,200,200,0.0)', 'yellow', 'green']


Figure(layout=FigureLayout(height='420px'))

In [15]:
no_ebike = stations[stations['nbebike'] == 0]
almost_no_ebike = stations[(stations['nbebike'] > 0) & (stations['nbebike'] < 4)]
ebikes = stations[stations['nbebike'] > 3]

In [16]:
scale = 3

no_ebike_layer = gmaps.symbol_layer(
    no_ebike[['lat', 'lng']], fill_color='red',
    stroke_color='red', scale=scale
)

almost_no_ebike_layer = gmaps.symbol_layer(
    almost_no_ebike[['lat', 'lng']], fill_color='yellow',
    stroke_color='yellow', scale=scale
)

ebikes_layer = gmaps.symbol_layer(
    ebikes[['lat', 'lng']], fill_color='green',
    stroke_color='green', scale=scale
)

info_box_template = """
<dl>
<dt>Name</dt><dd>{station_name}</dd>
<dt>Nb ebikes</dt><dd>{nbebike}</dd>
<dt>Nb free docks</dt><dd>{nbfreeedock}</dd>
</dl>
"""
station_info = [info_box_template.format(**station) for station in stations[['station_name', 'nbebike', 'nbfreeedock']].to_dict('records')]

marker_layer = gmaps.marker_layer(stations[['lat', 'lng']], info_box_content=station_info)

fig = gmaps.figure(center=paris_coordinates, zoom_level=12)

fig.add_layer(no_ebike_layer)
fig.add_layer(almost_no_ebike_layer)
fig.add_layer(ebikes_layer)
#fig.add_layer(marker_layer)

fig